# Analysing Historical Stock/Revenue Data

### Description:

It's very important for a Data Scientist to know how to collect, analyze and create visualizations with collected data for decision making.

This project aims to practice such skills, analyzing the stock data from Tesla and GameStop.

In these first two cells we install and import the relevant libraries for the project.

In [1]:
#!pip install yfinance
#!pip install pandas
#!pip install requests
#!pip install bs4
#!pip install plotly

In [2]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Define Graphing Function

In this section, we define the function make_graph. The make_graph function takes a dataframe with stock data (dataframe must contain Date and Close columns), a dataframe with revenue data (dataframe must contain Date and Revenue columns), and the name of the stock.

In [3]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data.Date, infer_datetime_format=True), y=stock_data.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data.Date, infer_datetime_format=True), y=revenue_data.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

## Question 1 - Extracting Tesla Stock Data Using yfinance

Using the Ticker function, we enter the ticker symbol of the stock we want to extract data on to create a ticker object. The stock is Tesla and its ticker symbol is TSLA.

Using the ticker object and the function history, we extract stock information and save it in a dataframe named tesla_data.

Then we set the period parameter to max so we get information for the maximum amount of time.

Now we reset the index using the reset_index(inplace=True) function on the tesla_data DataFrame and display the first five rows of the tesla_data dataframe using the head function.

In [4]:
Tesla = yf.Ticker('TSLA')

tesla_data = Tesla.history(period = "max")
tesla_data.reset_index(inplace = True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


## Question 2 - Extracting Tesla Revenue Data Using Webscraping

We'll use the requests library to download the webpage Tesla Revenue 2010-2021 | TSLA. Then save the text of the response as a variable named html_data.

Then we parse the html data using beautiful_soup.

In [5]:
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
html_data = requests.get(url).text

soup = BeautifulSoup(html_data, "html.parser")
soup.find_all('title')

[<title>Tesla Revenue 2010-2023 | TSLA | MacroTrends</title>]

Using beautiful soup we extract the table with Tesla Quarterly Revenue and store it into a dataframe named tesla_revenue.

For this project our dataframe will have columns Date and Revenue, and we'll remove the comma and dollar sign from the Revenue column and drop NaN values.

Here we'll display the last 5 row of the tesla_revenue dataframe using the tail function.

In [6]:
tesla_revenue = pd.read_html(str(soup.find_all('table')[1]),flavor='bs4')[0]
tesla_revenue.columns = ['Date','Revenue']

tesla_revenue["Revenue"] = tesla_revenue["Revenue"].str.replace("$", "").str.replace(",", "")

tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

tesla_revenue.tail()

,Date,Revenue
50,2010-09-30,31
51,2010-06-30,28
52,2010-03-31,21
54,2009-09-30,46
55,2009-06-30,27


## Question 3 - Extracting GameStop Stock Data Using yfinance

We'll repeat the process using the Ticker function. But this time we want GameStop stock data, and its ticker symbol is GME.

Using the ticker object and the function history, we extract stock information and save it in a dataframe named gme_data. Once again we set the period parameter to max so we get information for the maximum amount of time.

We reset the index using the reset_index(inplace=True) function on the gme_data DataFrame and display the first five rows of the gme_data dataframe using the head function.

In [7]:
GameStop = yf.Ticker("GME")
gme_data = GameStop.history(period = 'max')

gme_data.reset_index(inplace = True)
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683251,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658001,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


## Question 4 - Extracting GameStop Revenue Data Using Webscraping

We'll use the requests library to download the webpage GameStop Revenue 2010-2021 | GME. And save the text of the response as a variable named html_data.

Now we parse the html data using beautiful_soup.

In [8]:
url = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"
html_data = requests.get(url).text

soup = BeautifulSoup(html_data, "html.parser")
soup.find_all('title')

[<title>GameStop Revenue 2010-2023 | GME | MacroTrends</title>]

Using beautiful soup we extract the table with GameStop Quarterly Revenue and store it into a dataframe named gme_revenue.

For this project our dataframe will have columns Date and Revenue, and we'll remove the comma and dollar sign from the Revenue column.

We display the last five rows of the gme_revenue dataframe using the tail function.

In [9]:
gme_revenue = pd.read_html(str(soup.find_all('table')[1]),flavor='bs4')[0]
gme_revenue.columns = ['Date','Revenue']
gme_revenue["Revenue"] = gme_revenue["Revenue"].str.replace("$", "").str.replace(",", "")

gme_revenue.tail()

,Date,Revenue
52,2010-01-31,3524
53,2009-10-31,1835
54,2009-07-31,1739
55,2009-04-30,1981
56,2009-01-31,3492


## Question 5 - Tesla Stock and Revenue Dashboard

Now we're going to plot a graph with the information we colected, for this we'll use the make_graph function to graph the Tesla Stock Data. The structure to call the make_graph function is make_graph(tesla_data, tesla_revenue, 'Tesla')

In [ ]:
make_graph(tesla_data, tesla_revenue, 'Tesla')

## Question 6 - GameStop Stock and Revenue Dashboard

We'll repeat the last step here. The structure to call the make_graph function is make_graph(gme_data, gme_revenue, 'GameStop').

In [ ]:
make_graph(gme_data, gme_revenue, 'GameStop')

### Author: Ffion Davies